In [ ]:
import pandas as pd
import numpy as np
import os

# Cấu hình file
input_file = r"train_10GB_sorted.csv"
output_file = r"data_processed.csv"
chunksize = 5_000_000

# Pass 0: Đọc toàn bộ unique family để Label Encoding
print("Pass 0: Đang đọc unique family để Label Encoding...")

unique_families = set()
for chunk in pd.read_csv(input_file, usecols=['family'], chunksize=chunksize):
    unique_families.update(chunk['family'].unique())

family_mapping = {fam: idx for idx, fam in enumerate(sorted(unique_families))}
print(f"Đã tìm thấy {len(family_mapping)} nhóm family.")

# Pass 1: Tính min-max toàn bộ
global_min = {
    'sales': np.inf,
    'onpromotion': np.inf,
    'year': np.inf,
    'family_idx': np.inf,
    'store_idx': np.inf
}
global_max = {
    'sales': -np.inf,
    'onpromotion': -np.inf,
    'year': -np.inf,
    'family_idx': -np.inf,
    'store_idx': -np.inf
}

print("Pass 1: Đang tính min-max toàn bộ...")

for chunk in pd.read_csv(input_file, chunksize=chunksize, parse_dates=['date']):
    # Parse date
    chunk['year'] = chunk['date'].dt.year
    chunk['month'] = chunk['date'].dt.month
    chunk['day'] = chunk['date'].dt.day

    # Label Encoding
    chunk['family_idx'] = chunk['family'].map(family_mapping)

    # Tính min-max trên các cột
    for col in ['sales', 'onpromotion', 'year', 'family_idx', 'store_nbr']:
        # Đổi tên store_nbr thành store_idx luôn
        tmp_col = col if col != 'store_nbr' else 'store_idx'
        chunk[tmp_col] = chunk[col] if col != 'store_nbr' else chunk['store_nbr']

        global_min[tmp_col] = min(global_min[tmp_col], chunk[tmp_col].min())
        global_max[tmp_col] = max(global_max[tmp_col], chunk[tmp_col].max())

print("Min-max đã tính xong:")
for col in global_min.keys():
    print(f"{col}: min={global_min[col]}, max={global_max[col]}")

# Nếu file output đã tồn tại thì xoá trước
if os.path.exists(output_file):
    os.remove(output_file)

# Pass 2: Thực sự chuẩn hóa và lưu ra file
print("Pass 2: Đang chuẩn hóa và ghi file...")

first_chunk = True
for chunk in pd.read_csv(input_file, chunksize=chunksize, parse_dates=['date']):
    chunk['year'] = chunk['date'].dt.year
    chunk['month'] = chunk['date'].dt.month
    chunk['day'] = chunk['date'].dt.day
    chunk['family_idx'] = chunk['family'].map(family_mapping)
    chunk['store_idx'] = chunk['store_nbr']

    # Min-Max scaling
    for col in ['sales', 'onpromotion', 'year', 'family_idx', 'store_idx']:
        chunk[col + '_scaled'] = (chunk[col] - global_min[col]) / (global_max[col] - global_min[col])

    # Cyclical encoding
    chunk['month_sin'] = np.sin(2 * np.pi * chunk['month'] / 12)
    chunk['month_cos'] = np.cos(2 * np.pi * chunk['month'] / 12)
    chunk['day_sin'] = np.sin(2 * np.pi * chunk['day'] / 31)
    chunk['day_cos'] = np.cos(2 * np.pi * chunk['day'] / 31)

    final_cols = [
        'sales_scaled', 'onpromotion_scaled', 'year_scaled',
        'month_sin', 'month_cos', 'day_sin', 'day_cos',
        'family_idx_scaled', 'store_idx_scaled'
    ]

    # Rename về family_scaled, store_scaled theo chuẩn bạn mong muốn
    chunk.rename(columns={
        'family_idx_scaled': 'family_scaled',
        'store_idx_scaled': 'store_scaled'
    }, inplace=True)

    chunk[[
        'sales_scaled', 'onpromotion_scaled', 'year_scaled',
        'month_sin', 'month_cos', 'day_sin', 'day_cos',
        'family_scaled', 'store_scaled'
    ]].to_csv(output_file, mode='w' if first_chunk else 'a', header=first_chunk, index=False)
    first_chunk = False

print("✅ Tiền xử lý hoàn tất. File đã lưu tại:", output_file)
